In [16]:
import numpy as np
import time
from LPM import LPM
import utils
import params

In [3]:
lpm = LPM('quadrotor_linear_planning_model.mat')

In [9]:
k = np.array([[0.0, 0.0, 1.0], [0.0, 0.0, 1.0]])
start_t = time.time()
lpm.compute_trajectory(k)
print("elapsed time: ", time.time() - start_t)

elapsed time:  0.3845367431640625


In [17]:
n_plan_max = 10000

v_0 = np.zeros((params.N_DIM,1))

v_max = 2.0 # L1 velocity constraints
v_max_norm = 2.0
V_bounds = np.tile(np.array([-v_max, v_max]), (1,params.N_DIM))[0]

start_t = time.time()
V_peak = utils.rand_in_bounds(V_bounds, n_plan_max)

# eliminate samples that exceed the max velocity and max delta
# from initial velocity
V_peak_mag = np.linalg.norm(V_peak, ord=2, axis=0)
delta_V_peak_mag = np.linalg.norm(V_peak - v_0, ord=2, axis=0)
V_peak_test = V_peak_mag < v_max_norm
V_peak = V_peak[:,V_peak_test]
# V_peak_mag = np.linalg.norm(V_peak, 2, 0)
# delta_V_peak_mag = np.linalg.norm(V_peak - np.tile(self.v_0,(1,self.n_plan_max)), 2, 0)
# V_peak_test = np.logical_and(V_peak_mag < self.v_max_norm, delta_V_peak_mag < self.delta_v_peak_max)
# V_peak = V_peak[:,V_peak_test]
print("elapsed time: ", time.time() - start_t)

elapsed time:  0.001001119613647461


In [24]:
np.tile(np.array([-v_max, v_max]), (1,params.N_DIM))

array([[-2.,  2., -2.,  2.]])

In [22]:
V_peak

array([[-0.17690017,  0.78355997,  1.02657601, ...,  0.40967189,
         0.90227219, -0.472961  ],
       [ 0.09663823, -0.23277067,  0.53018277, ..., -1.58534974,
         1.62128416,  1.81971359]])